In [42]:
from typing import List
import math
from functools import cmp_to_key

        
def distance_to_line(self, p1, p2):
    x_diff = p2[0] - p1[0]
    y_diff = p2[1] - p1[1]
    P_x = self[0] - p1[0]
    P_y = self[1] - p1[1]
    num = abs(y_diff*P_x - x_diff*P_y )
    den = math.sqrt(y_diff**2 + x_diff**2)
    return num / den
    
def subtract(self, p):
    return [self[0] - p[0], self[1] - p[1]]
    
def cross_product(p1, p2):
    return p1[0] * p2[1] - p2[0] * p1[1]

def direction(p1, p2, p3):
    return  cross_product(subtract(p3,p1), subtract(p2,p1))

def distance_sq(p1, p0): 
    return (p1[0]-p0[0])**2 + (p1[1]-p0[1])**2
    
    
# find the point with minimum y coordinate
# in case of tie choose the point with minimun x-coordinate
def find_min_y(points):
    miny = 999999
    mini = 0
    for i, point in enumerate(points):
        if point[1] < miny:
            miny = point[1]
            mini = i
        if point[1] == miny:
            if point[0] < points[mini][0]:
                mini = i
    return points[mini], mini

# comparator for the sorting 
def polar_comparator(p1, p2, p0):
    d = direction(p0, p1, p2)
    if d < 0:
        return -1
    if d > 0:
        return 1
    if d == 0:
        if distance_sq(p1, p0) < distance_sq(p2, p0):
            return -1
        else:
            return 1

def graham_scan(points):
    # let p0 be the point with minimum y-coordinate,
    # or the leftmost such point in case of a tie
    p0, index = find_min_y(points)

    # swap p[0] with p[index]
    points[0], points[index] = points[index], points[0]

    # sort the points (except p0) according to the polar angle
    # made by the line segment with x-axis in anti-clockwise direction
    sorted_polar = sorted(points[1:], key=cmp_to_key ( lambda p1, p2: polar_comparator(p1, p2, p0)))
    
    
    # if more than two points are collinear with p0, keep the farthest
    to_remove = []
    for i in range(len(sorted_polar) - 1):
        d = direction(sorted_polar[i], sorted_polar[i + 1], p0)
        if d == 0:
            to_remove.append(i)
    sorted_polar = [i for j, i in enumerate(sorted_polar) if j not in to_remove]

   
    m = len(sorted_polar)
    if m < 2:
        return 0

    else:
        stack = []
        stack_size = 0
        stack.append(points[0])
        stack.append(sorted_polar[0])
        stack.append(sorted_polar[1])
        stack_size = 3

        for i in range(2, m):
            while (True):
                d = direction(stack[stack_size - 2], stack[stack_size - 1], sorted_polar[i])
                if d < 0: # if it makes left turn
                    break
                else: # if it makes non left turn
                    stack.pop()
                    stack_size -= 1
            stack.append(sorted_polar[i])
            stack_size += 1
    return stack


        
class Airport:
    def airport(self, houses: List[List[int]]) -> float:
        """
        Find the best place to build airport and
        calculate the average distance from all the house to airport

        Parameters:
            houses(list[list[int]]): List of houses.
                Each house contains [x,y] coordination.

        Returns:
            distance(float)
        """
    
        ans = graham_scan(houses)
        
        if ans==0:
            return 0
        
        addx = 0
        addy = 0
        eqlen = len(houses)
        for i in houses :
            addx = addx + i[0]
            addy = addy + i[1]
        eqpoint = [addx/eqlen,addy/eqlen]
        
        
        
        min_distance=9999999999999
        
        for i in range(len(ans)-1):
            total_distance = distance_to_line(eqpoint,ans[i],ans[i+1])
            if total_distance < min_distance :
                min_distance=total_distance
        #line between last point and start point 
        total_distance = distance_to_line(eqpoint,ans[0],ans[-1])

        if total_distance < min_distance :
            min_distance=total_distance
        
        
        return min_distance

 

In [43]:
ans=Airport().airport([[1,1],[2,2],[0,2],[2,0],[2,4],[3,3],[4,2],[4,1],[4,0]])

In [44]:
ans

1.3356461422412562

In [45]:
if __name__ == "__main__":
    print(Airport().airport([[0,0],[1,0]]))
    """
    0.0
    """
    print(Airport().airport([[0,0],[1,0],[0,1]]))
    """
    *.
    **
    # Convex: [[0, 0], [1, 0], [0, 1]]
    0.2357022603955159
    """
    print(Airport().airport([[0,0],[2,0],[0,2],[1,1],[2,2]]))
    """
    *.*
    .*.
    *.*
    # Convex: [[0, 0], [2, 0], [2, 2], [0, 2]]
    1.0
    """
    print(Airport().airport([[1,1],[2,2],[0,2],[2,0],[2,4],[3,3],[4,2],[4,1],[4,0]]))
    """
    ..*..
    ...*.
    *.*.*
    .*..*
    ..*.*
    # Convex: [[0, 2], [2, 0], [4, 0], [4, 2], [2, 4]]
    1.3356461422412562
    """

0
0.2357022603955159
1.0
1.3356461422412562
